# <h1><center>Sales Analysis and Visualization</center></h1>

## Outline

This notebook provides an analysis on retail and warehouse sales. The data contains 307,645 sales records and 9 columns, including retail sales, warehouse sales, name of suppliers, and item descriptions.

The cleaning and formatting process includes:

- filling missing data
- checking fior duplicate records
- dropping highly correlated columns
- formatting the date

The analysis answers a variety of business questions such as:

- Which item sold the most in quantity?
- What is the overall sales trend per month and per year?
- Which suppliers had the most profit in retail and warehouse sales?
- Which item types had the most profit in retail and warehouse sales?
- Which item had the most profit in retail and warehouse sales?

Most of the data visualizations are done using Plotly express because of their interactive capabilities. Seaborn is also used to create heatmaps.

---

### What is the source of the data?

#### This data was acquired at: https://catalog.data.gov/dataset/warehouse-and-retail-sales

### Import libraries

In [ ]:
import pandas as pd
import os
import seaborn as sns

In [ ]:
pwd = os.getcwd()

In [ ]:
sales_data = pd.read_csv(pwd + "\\Warehouse_and_Retail_Sales.csv")

### What is the size of the dataset?

In [ ]:
shape = sales_data.shape
print(f"There are {shape[0]} rows and {shape[1]} columns.")
print(f"{shape[0]*shape[1]} total elements.")

### What are the columns of the dataset?

In [ ]:
list(sales_data.columns)

### What are the data types of the column variables?

In [ ]:
sales_data.info()

### Is there missing data in the dataset?

In [ ]:
sales_data.isnull().sum()

In [ ]:
print(f"There is {round(sales_data.isnull().sum().sum()/2768805,4)}% missing data in this dataset.")

### Create a filter that only shows the rows that have missing values

#### Missing values for "SUPPLIER"

In [ ]:
supplier_nan = pd.isnull(sales_data["SUPPLIER"])
sales_data[supplier_nan]

#### Missing values for "ITEM TYPE"

In [ ]:
item_type_nan = pd.isnull(sales_data["ITEM TYPE"])
sales_data[item_type_nan]

#### Missing values for "RETAIL SALES"

In [ ]:
retail_nan = pd.isnull(sales_data["RETAIL SALES"])
sales_data[retail_nan]

### Fill missing data, impute mode values for categorical data and mean values for numerical data

In [ ]:
sales_data["SUPPLIER"].fillna(sales_data["SUPPLIER"].mode().iloc[0], inplace=True)

sales_data["ITEM TYPE"].fillna(sales_data["ITEM TYPE"].mode().iloc[0], inplace=True)

sales_data["RETAIL SALES"].fillna(sales_data["RETAIL SALES"].mean(), inplace=True)

sales_data.isnull().sum()

### Check on rows that previously had missing data to ensure the imputation process worked

#### SUPPLIER imputation check

In [ ]:
sales_data[supplier_nan]

#### RETAIL SALES imputation check

In [ ]:
sales_data[retail_nan]

#### ITEM TYPE imputation check

In [ ]:
sales_data[item_type_nan]

### Duplicates (no duplicate records found)

In [ ]:
sales_data[sales_data.duplicated()]

### Exploring Feature Correlations: Heatmap Analysis

In [ ]:
sns.heatmap(sales_data.corr(),annot = True)

#### Retail Transfers and Retail Sales are highly correlated (96%), let's drop Retail Transfers since we're more curious about the sales

In [ ]:
sales_data.drop(columns= "RETAIL TRANSFERS", inplace = True)
sales_data.head()

### Data Formatting

#### Formatting the date

In [ ]:
#Format the new date in new column
sales_data["DATE"] = pd.to_datetime(sales_data["MONTH"].astype(str) + ' ' + sales_data["YEAR"].astype(str), format='%m %Y')

#Drop old date columns
sales_data.drop(columns = ["YEAR", "MONTH"], inplace = True)

#Reorder columns
sales_data = sales_data.loc[:,["DATE", "SUPPLIER", "ITEM CODE", "ITEM DESCRIPTION", "ITEM TYPE", "RETAIL SALES", "WAREHOUSE SALES"]]
sales_data.head()

#### Sorting Data by date

In [ ]:
sd_sorted_date = sales_data.sort_values("DATE")
sales_data  = sd_sorted_date
sales_data.head()

### Univariate Analysis

In [ ]:
import plotly.express as px


#### Visualization of most occuring suppliers descending from most to least 

In [ ]:
fig1 = px.histogram(sales_data, x="SUPPLIER").update_xaxes(categoryorder='total descending')
fig1.show()

#### Who are the top 5 occurring suppliers?

In [ ]:
print("Top 5 suppliers who sold the most: ")
sales_data["SUPPLIER"].value_counts()[0:5]

#### Bottom 5 occuring suppliers?

In [ ]:
print("Top 5 suppliers who sold the least: ")
sales_data["SUPPLIER"].value_counts()[len(sales_data["SUPPLIER"].value_counts())-5:len(sales_data["SUPPLIER"].value_counts())]

#### Which item sold the most in quantity?

In [ ]:
fig2 = px.histogram(sales_data, x="ITEM DESCRIPTION").update_xaxes(categoryorder='total descending')
fig2.update_layout(bargap=.5)
fig2.show()

#### Top 5 items sold

In [ ]:
print("Top 5 items sold: ")
sales_data["ITEM DESCRIPTION"].value_counts()[0:5]

####  Bottom 5 items sold

In [ ]:
print("Bottom 5 items sold: ")
sales_data["ITEM DESCRIPTION"].value_counts()[len(sales_data["ITEM DESCRIPTION"].value_counts())-5:len(sales_data["ITEM DESCRIPTION"].value_counts())]

#### Which item type sold the most in quantity?

In [ ]:
fig3 = px.histogram(sales_data, x="ITEM TYPE").update_xaxes(categoryorder='total descending')
fig3.update_layout(bargap=.5)
fig3.show()

#### Top 4 item types sold

In [ ]:
print("Top 4 item types sold: ")
sales_data["ITEM TYPE"].value_counts()[0:4]

#### Bottom 4 item types sold

In [ ]:
print("Bottom 4 items least sold: ")
sales_data["ITEM TYPE"].value_counts()[len(sales_data["ITEM TYPE"].value_counts())-4:len(sales_data["ITEM TYPE"].value_counts())]

#### Statistics on retail sales and warehouse sales

In [ ]:
sales_data["RETAIL SALES"].describe()

In [ ]:
sales_data["WAREHOUSE SALES"].describe()

## Bivariate Analysis

### What is the overall sales trend per month?

#### Group the retail and warehouse sales by month and sum them

In [ ]:
sales_per_month = sales_data.groupby(pd.Grouper(key='DATE', freq='M')).sum()
retail_per_month = sales_per_month["RETAIL SALES"]

ware_per_month = sales_per_month["WAREHOUSE SALES"]
month = sales_per_month.index

#### Retail/Warehouse Visualization

In [ ]:
fig2 = px.line(sales_per_month, x=month, y=[retail_per_month,ware_per_month])

fig2.update_layout(width=1000, title={
    "text": "Retail/Warehouse Sales per Month (4 Years)",
    "x": 0.5,
    "y": 0.95,
    "xanchor": "center",
    "yanchor": "top"
}, 
    xaxis_title = "Month",
    yaxis_title="Total Sales"
                  )
fig2.show()

### What is the overall sales trend per year?

#### Group the retail and warehouse sales by year and sum them

In [ ]:
sales_per_year = sales_data.groupby(pd.Grouper(key='DATE', freq='Y')).sum()
retail_per_year = sales_per_year["RETAIL SALES"]

ware_per_year = sales_per_year["WAREHOUSE SALES"]
year = sales_per_year.index

#### Retail/Warehouse Visualization

In [ ]:
fig3 = px.line(sales_per_year, x=year, y=[retail_per_year,ware_per_year])

fig3.update_layout(width=50, title={
    "text": "Retail/Warehouse Sales Per Year",
    "x": 0.5,
    "y": 0.95,
    "xanchor": "center",
    "yanchor": "top"
}, 
    xaxis_title = "Time",
    yaxis_title="Total Sales"
                  )
fig3.show()

### Which suppliers had the most profit in retail sales?

In [ ]:
rsupplier_sales_data = sales_data.groupby("SUPPLIER")["RETAIL SALES"].sum().sort_values(ascending = False)
wsupplier_sales_data = sales_data.groupby("SUPPLIER")["WAREHOUSE SALES"].sum().sort_values(ascending = False)

rsupplier = rsupplier_sales_data.index
rsupplier_sales = rsupplier_sales_data.values

wsupplier = wsupplier_sales_data.index
wsupplier_sales = wsupplier_sales_data.values

#There are numerous suppliers so let's only focus on the top 20%
percent = .2
cutoff = int(round(percent * len(rsupplier_sales_data),0))
rsupplier = rsupplier[0:cutoff]
rsupplier_sales = rsupplier_sales[0:cutoff]

wsupplier = wsupplier[0:cutoff]
wsupplier_sales = wsupplier_sales[0:cutoff]

In [ ]:
fig4 = px.bar(x=rsupplier, y=rsupplier_sales)

fig4.update_layout(
    title='Supplier Retail Sales (Top 20%)',
    xaxis_title='Suppliers',
    yaxis_title='Retail Sales'
)

fig4.show()

In [ ]:
print(f'Top 5 Suppliers by Retail Sales: \n{rsupplier_sales_data[0:5]}')

In [ ]:
print(f'Bottom 5 Suppliers by Retail Sales: \n{rsupplier_sales_data[len(rsupplier_sales_data)-5:len(rsupplier_sales_data)]}')

### Which suppliers had the most profit in warehouse sales?

In [ ]:
fig5 = px.histogram(x=wsupplier, y=wsupplier_sales)

fig5.update_layout(
    title='Supplier Warehouse Sales (Top 20%)',
    xaxis_title='Suppliers',
    yaxis_title='Warehouse Sales'
)

fig5.show()

In [ ]:
print(f'Top 5 Suppliers by Warehouse Sales: \n{wsupplier_sales_data[0:5]}')

In [ ]:
print(f'Bottom 5 Suppliers by Warehouse Sales: \n{wsupplier_sales_data[len(wsupplier_sales_data)-5:len(wsupplier_sales_data)]}')

#### Which item type had the most profit in retail sales?

In [ ]:
rtype_sales_data = sales_data.groupby("ITEM TYPE")["RETAIL SALES"].sum().sort_values(ascending = False)
rtype = rtype_sales_data.index
rtype_sales = rtype_sales_data.values

In [ ]:
fig4 = px.bar(x=rtype, y=rtype_sales)

fig4.update_layout(
    title='Item Type Retail Sales',
    xaxis_title='Item Type',
    yaxis_title='Retail Sales'
)

fig4.show()

#### Which item type had the most profit in warehouse sales?

In [ ]:
wtype = wtype_sales_data.index
wtype_sales = wtype_sales_data.values
wtype_sales_data = sales_data.groupby("ITEM TYPE")["WAREHOUSE SALES"].sum().sort_values(ascending = False)

In [ ]:
fig5 = px.histogram(x=wtype, y=wtype_sales)

fig5.update_layout(
    title='Item Type Warehouse Sales',
    xaxis_title='Item Type',
    yaxis_title='Warehouse Sales'
)

fig5.show()

#### Which item had the most profit in retail sales? Warehouse sales?

In [ ]:
rdesc_sales_data = sales_data.groupby("ITEM DESCRIPTION")["RETAIL SALES"].sum().sort_values(ascending = False)
wdesc_sales_data = sales_data.groupby("ITEM DESCRIPTION")["WAREHOUSE SALES"].sum().sort_values(ascending = False)

#There are a lot of Item Descriptions, difficult for visualization to handle. This will cut it down to 1% of the data
percent = .01
cutoff = int(round(percent * len(rdesc_sales_data),0))

rdesc_sales_cut = rdesc_sales_data[0:cutoff]
wdesc_sales_cut = wdesc_sales_data[0:cutoff]

#Assign 1% of item description data to rdesc
rdesc = rdesc_sales_cut.index

#Assign retail sales to redesc_sales
rdesc_sales = list(rdesc_sales_cut)

#supplier_rsales_data[supplier_rsales_data.values > 1]

wdesc = wdesc_sales_cut.index
wdesc_sales = wdesc_sales_cut.values

In [ ]:
fig5 = px.bar(x=rdesc, y=rdesc_sales)

fig5.update_layout(
    title='Item Description by Retail Sales (Top 1%)',
    xaxis_title='Item Description',
    yaxis_title='Retail Sales'
)

fig5.show()

In [ ]:
print(f'Top 5 Item Descriptions by Retail Sales: \n{rdesc_sales_data[0:5]}')

In [ ]:
print(f'Bottom 5 Item Descriptions by Retail Sales: \n{rdesc_sales_data[len(rdesc_sales_data)-5:len(rdesc_sales_data)]}')

In [ ]:
fig5 = px.bar(x=wdesc, y=wdesc_sales)

fig5.update_layout(
    title='Item Description by Warehouse Sales (Top 1%)',
    xaxis_title='Item Description',
    yaxis_title='Warehouse Sales'
)

fig5.show()

In [ ]:
print(f'Top 5 Item Descriptions by Warehouse Sales: \n{wdesc_sales_data[0:5]}')

In [ ]:
print(f'Bottom 5 Item Descriptions by Warehouse Sales: \n{wdesc_sales_data[len(wdesc_sales_data)-5:len(wdesc_sales_data)]}')

### Thank for looking through this notebook! 
#### Any questions, comments, or suggestions would be greatly appreciated!